# Decision tree

**Ví dụ cho bài toán đưa ra quyết định học hay chơi:**

Nếu: 

- ngày còn lại trước khi thi > 2 => đi chơi
- ngày còn lại trước khi thi <2 và có trận bóng tối hôm đó => chơi
- các trường hợp còn lại là học

=> Ta có decision tree

![](../img/dt_1.png)

**Ví dụ khác cho decision tree cho bài toán classifier**

![](../img/dt_2.png)

Ta thấy rằng việc tìm ra đường phân cách giữa 2 class trên là những đường thẳng song song với các trục tọa độ.

Các điểm thuộc class màu đỏ có đặc tính là:

- t3 < x1 < t1
- x2 < t2


Với decision tree trên ta có các ô màu xám, lục đỏ gọi là `node`. Các `node` thể hiện đầu ra màu lục, đỏ gọi là `leaf node` hoặc `terminal node`. Các node thể hiện câu hỏi màu xám là `non-leaf node`. `non-leaf node` đầu tiên là node gốc hay `root node`. Các `non-leaf node` thường có hai hoặc nhiều node con (`child node`). Các `child node` này có thể là một `leaf node` hoặc một `non-leaf node` khác. Các `child node` có cùng bố mẹ được gọi là `sibling node`. Nếu tất cả các `non-leaf node` chỉ có hai `child node`, ta nói rằng đó là một `binary decision tree` (cây quyết định nhị phân). Các câu hỏi trong `binary decision tree` đều có thể đưa được về dạng câu hỏi đúng hay sai. Các decision tree mà một leaf node có nhiều child node cũng có thể được đưa về dạng một binary decision tree. Điều này có thể đạt được vì hầu hết các câu hỏi đều có thể được đưa về dạng câu hỏi đúng sai.

## I. ID3

ID3 là một thuật toán decision tree được áp dụng cho các bài toán classification mà tất cả các thuộc tính đều ở dạng categorical. Trong bài tiếp theo, chúng ta sẽ làm quen với một thuật toán khác có tên là Classification and Regression Tree (CART)–có thể được áp dụng vào cả hai loại classification và regression, như tên gọi của nó–làm việc với cả thuộc tính dạng categorical và liên tục.

### Ví dụ:

Cho bảng mô tả mối quan hệ của thời tiết qua 4 cột data trong 14 ngày và quyết định tổ chức sự kiện `play` là có hay không.

Yêu cầu đưa ra là phải dự đoán giá trị ở cột `play` khi biết giá trị của 4 cột thời tiết.

| id | outlook  | temperature | humidity | wind   | play |
|----|----------|-------------|----------|--------|------|
| 1  | sunny    | hot         | high     | weak   | no   |
| 2  | sunny    | hot         | high     | strong | no   |
| 3  | overcast | hot         | high     | weak   | yes  |
| 4  | rainy    | mild        | high     | weak   | yes  |
| 5  | rainy    | cool        | normal   | weak   | yes  |
| 6  | rainy    | cool        | normal   | strong | no   |
| 7  | overcast | cool        | normal   | strong | yes  |
| 8  | sunny    | mild        | high     | weak   | no   |
| 9  | sunny    | cool        | normal   | weak   | yes  |
| 10 | rainy    | mild        | normal   | weak   | yes  |
| 11 | sunny    | mild        | normal   | strong | yes  |
| 12 | overcast | mild        | high     | strong | yes  |
| 13 | overcast | hot         | normal   | weak   | yes  |
| 14 | rainy    | mild        | high     | strong | no   |


Có 4 thuộc tính thời tiết:

1. `Outlook` nhận một trong ba giá trị: `sunny, overcast, rainy`
2. `Temperature` nhận một trong ba giá trị: `hot, cool, mild`
3. `Humidity` nhận một trong hai giá trị: `high, normal`.
4. `Wind` nhận một trong hai giá trị: `weak, strong`

(Tổng cộng có 3×3×2×2=36 loại thời tiết khác nhau, trong đó 14 loại được thể hiện trong bảng.)

Đây có thể được coi là một bài toán dự đoán liệu đội bóng có chơi bóng không dựa trên các quan sát thời tiết. Ở đây, các quan sát đều ở dạng categorical. Cách dự đoán dưới đây tương đối đơn giản và khá chính xác, có thể không phải là cách ra quyết định tốt nhất:

- Nếu outlook = sunny và humidity = high thì play = no.

- Nếu outlook = rainy và windy = true thì play = no.

- Nếu outlook = overcast thì play = yes.

- Ngoài ra, nếu humidity = normal thì play = yes.

- Ngoài ra, play = yes.


**Tìm thứ tự các thuộc tính trong decision tree với ID3:**

### Điều kiện dừng

Trong các thuật toán decision tree nói chung và ID3 nói riêng, nếu ta tiếp tục phân chia các node chưa tinh khiết, ta sẽ thu được một tree mà mọi điểm trong tập huấn luyện đều được dự đoán đúng (giả sử rằng không có hai input giống nhau nào cho output khác nhau). Khi đó, tree có thể sẽ rất phức tạp (nhiều node) với nhiều leaf node chỉ có một vài điểm dữ liệu. Như vậy, nhiều khả năng overfitting sẽ xảy ra.

Để tránh overfitting, một trong số các phương pháp sau có thể được sử dụng. Tại một node, nếu một trong số các điều kiện sau đây xảy ra, ta không tiếp tục phân chia node đó và coi nó là một leaf node:
- nếu node đó có entropy bằng 0, tức mọi điểm trong node đều thuộc một class.

- nếu node đó có số phần tử nhỏ hơn một ngưỡng nào đó. Trong trường hợp này, ta chấp nhận có một số điểm bị phân lớp sai để tránh overfitting. Class cho leaf node này có thể được xác định dựa trên class chiếm đa số trong node.

- nếu khoảng cách từ node đó đến root node đạt tới một giá trị nào đó. Việc hạn chế chiều sâu của tree này làm giảm độ phức tạp của tree và phần nào giúp tránh overfitting.

- nếu tổng số leaf node vượt quá một ngưỡng nào đó.

- nếu việc phân chia node đó không làm giảm entropy quá nhiều (information gain nhỏ hơn một ngưỡng nào đó).

Ngoài các phương pháp trên, một phương pháp phổ biến khác được sử dụng để tránh overfitting là pruning, tạm dịch là cắt tỉa.

## II. Pruning


